In [119]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              16
On-line CPU(s) list: 0-15
Thread(s) per core:  1
Core(s) per socket:  1
Socket(s):           16
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               44
Model name:          Westmere E56xx/L56xx/X56xx (Nehalem-C)
Stepping:            1
CPU MHz:             2127.966
BogoMIPS:            4255.93
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            4096K
NUMA node0 CPU(s):   0-15
Flags:               fpu de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 syscall nx lm constant_tsc rep_good nopl pni pclmulqdq ssse3 cx16 sse4_1 sse4_2 x2apic popcnt aes hypervisor lahf_lm


In [120]:
import colorama
from colorama import Fore
from itertools import chain
import matplotlib.pyplot as plt
from giverny.isotropic_cube import *
from giverny.turbulence_toolkit import *
from giverny.turbulence_gizmos.basic_gizmos import *

# user-defined parameters for instantiating the isotropic cube.
# -----
# turbulence dataset name, e.g. "isotropic8192" or "isotropic1024fine".
cube_title = 'isotropic8192'

In [167]:
from numba import jit, njit, prange

In [122]:
# generates the morton cube representing the turbulence dataset.
cube = iso_cube(cube_title = cube_title)

In [174]:
# define functions

# build 16*16*16 bucket
def formBucket(voxel_point_value, num_values_per_datapoint, Bucket_length = 16, ):
    group_voxel = np.zeros((num_values_per_datapoint,Bucket_length,Bucket_length,Bucket_length))
    #Change from ZYX to XYZ, using transpose .T
    group_voxel[:,0:8,0:8,0:8] = voxel_point_value[0].T
    group_voxel[:,8:16,0:8,0:8] = voxel_point_value[1].T
    group_voxel[:,0:8,8:16,0:8] = voxel_point_value[2].T
    group_voxel[:,8:16,8:16,0:8] = voxel_point_value[3].T
    group_voxel[:,0:8,0:8,8:16] = voxel_point_value[4].T
    group_voxel[:,8:16,0:8,8:16] = voxel_point_value[5].T
    group_voxel[:,0:8,8:16,8:16] = voxel_point_value[6].T
    group_voxel[:,8:16,8:16,8:16] = voxel_point_value[7].T
    
    return group_voxel

# find center coordinates of the bucket (e.g, (7.5 7.5 7.5) for the first bucket)
def findCenter(points, dx):
    center_point = np.array([points[0], points[1], points[2]])/dx%8
    for i in range(len(center_point)):
        if (center_point[i] < 3.5):
            center_point[i]=center_point[i]+8
    return center_point

def findCenter_gradient(points, dx):
    points = np.round(np.around(points/dx, decimals=1))
#    print('points',points)
    center_point = np.array([points[0], points[1], points[2]])%8
#    print('center_point',center_point)
    for i in range(len(center_point)):
        if (center_point[i] < 3.5):
            center_point[i]=center_point[i]+8
    return center_point

################################################################################
# Coefficients for Lagrange interpolation, 4,6,8th-order
################################################################################

def getLag4C(fr):
    #------------------------------------------------------
    # get the 1D vectors for the 8 point Lagrange weights
    # inline the constants, and write explicit for loop
    # for the C compilation
    #------------------------------------------------------
    #cdef int n
    wN = [1.,-3.,3.,-1.]
    g  = np.array([0,1.,0,0])
    #----------------------------
    # calculate weights if fr>0, and insert into gg
    #----------------------------
    if (fr>0):
        s = 0
        for n in range(4):
            g[n] = wN[n]/(fr-n+1)
            s += g[n]
        for n in range(4):
            g[n] = g[n]/s
            
    return g

################################################################################
# Functions for Lagrange interpolation, 4,6,8th-order
################################################################################

def interpLag4C(p,u):
    #--------------------------------------------------------
    # p is an np.array(3) containing the three coordinates
    #---------------------------------------------------------
    # get the coefficients
    #----------------------
    ix = p.astype('int')
    fr = p-ix
    gx = getLag4C(fr[0])
    gy = getLag4C(fr[1])
    gz = getLag4C(fr[2])
    #------------------------------------
    # create the 3D kernel from the 
    # outer product of the 1d kernels
    #------------------------------------
    gk = np.einsum('i,j,k',gx,gy,gz)
    #---------------------------------------
    # assemble the 4x4x4 cube and convolve
    #---------------------------------------
    d = u[:,ix[0]-1:ix[0]+3,ix[1]-1:ix[1]+3,ix[2]-1:ix[2]+3]
    ui = np.einsum('ijk,lijk->l',gk,d)
    
    return ui




################################################################################
# Coefficients for Hessian (central differencing), 4,6,8th-order
################################################################################   
@jit(nopython=True)
def getNone_Fd4_diagonal(dx):
    CenteredFiniteDiffCoeff = [( -1.0 / 12.0 / dx / dx, 4.0 / 3.0 / dx / dx, -15.0 / 6.0 / dx / dx,
                                4.0 / 3.0 / dx / dx, -1.0 / 12.0 / dx / dx)]
    return CenteredFiniteDiffCoeff

@jit(nopython=True)
def getNone_Fd4_offdiagonal(dx):
    CenteredFiniteDiffCoeff = [(-1.0 / 48.0 / dx / dx,1.0 / 48.0 / dx / dx,-1.0 / 48.0 / dx / dx,1.0 / 48.0 / dx / dx,
                               1.0 / 3.0 / dx / dx, -1.0 / 3.0 / dx / dx,1.0 / 3.0 / dx / dx, -1.0 / 3.0 / dx / dx)]
    return CenteredFiniteDiffCoeff
                

#######################################
# glue adjacent cornercode
def ranges(nums):
    nums = sorted(set(nums))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s + 512 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))

# same as def get_file_for_point
def findPath(cube, cornercode):
    t = cube.cache[(cube.cache['minLim'] <= cornercode) & (cube.cache['maxLim'] >= cornercode)]
    t = t.iloc[0]
    db_minLim = t.minLim 
    db_maxLim = t.maxLim
    path = cube.filepaths[f'{t.ProductionDatabaseName}_{var}_{timepoint}']
    
    return db_minLim, db_maxLim, path

def Lag_looKuptable_4(NB):
    frac = np.linspace(0,1-1/NB,NB)
    LW = []
    for fp in frac:
        LW.append(getLag4C(fp))
        
    return LW


# https://numba.discourse.group/t/passing-a-list-of-numpy-arrays-into-np-array-with-numba/278
@njit(parallel=True)
def make_2d(arraylist):
    n = len(arraylist)
    k = arraylist[0].shape[0]
    a2d = np.zeros((n, k))
    for i in prange(n):
        a2d[i] = arraylist[i]
    return(a2d)


@jit(nopython=True)#, parallel=True)
def HessianNone_Fd4(p,u,dx):
    #--------------------------------------------------------
    # p is an np.array(3) containing the three coordinates
    #---------------------------------------------------------
    # get the coefficients
    #----------------------
    ix = p.astype(np.int64)
    CenteredFiniteDiffCoeff_dia = np.array(getNone_Fd4_diagonal(dx))
    CenteredFiniteDiffCoeff_offdia = np.array(getNone_Fd4_offdiagonal(dx))
    #---------------------------------------
    # assemble the 5x5x5 cube and convolve
    #---------------------------------------
    # diagnoal components
    component_x = u[:,ix[0]-2:ix[0]+3,ix[1],ix[2]]
    component_y = u[:,ix[0],ix[1]-2:ix[1]+3,ix[2]]
    component_z = u[:,ix[0],ix[1],ix[2]-2:ix[2]+3]
    
#     uii = np.inner(CenteredFiniteDiffCoeff_dia,component_x)
    uii = np.dot(CenteredFiniteDiffCoeff_dia,component_x.T)
#     print("Dot vs. inner: ", np.all(np.dot(CenteredFiniteDiffCoeff_dia,component_x.T) == uii))
#     ujj = np.inner(CenteredFiniteDiffCoeff_dia,component_y)
    ujj = np.dot(CenteredFiniteDiffCoeff_dia,component_y.T)
#     ukk = np.inner(CenteredFiniteDiffCoeff_dia,component_z)
    ukk = np.dot(CenteredFiniteDiffCoeff_dia,component_z.T)
    
    # off-diagnoal components
    component_xy = [u[:,ix[0]+2,ix[1]+2,ix[2]],u[:,ix[0]+2,ix[1]-2,ix[2]],u[:,ix[0]-2,ix[1]-2,ix[2]],u[:,ix[0]-2,ix[1]+2,ix[2]],
                    u[:,ix[0]+1,ix[1]+1,ix[2]],u[:,ix[0]+1,ix[1]-1,ix[2]],u[:,ix[0]-1,ix[1]-1,ix[2]],u[:,ix[0]-1,ix[1]+1,ix[2]]]
#     print("component_xy: ", component_xy)
    component_xy = make_2d(component_xy)
    component_xz = [u[:,ix[0]+2,ix[1],ix[2]+2],u[:,ix[0]+2,ix[1],ix[2]-2],u[:,ix[0]-2,ix[1],ix[2]-2],u[:,ix[0]-2,ix[1],ix[2]+2],
                    u[:,ix[0]+1,ix[1],ix[2]+1],u[:,ix[0]+1,ix[1],ix[2]-1],u[:,ix[0]-1,ix[1],ix[2]-1],u[:,ix[0]-1,ix[1],ix[2]+1]]
    component_xz = make_2d(component_xz)
    component_yz = [u[:,ix[0],ix[1]+2,ix[2]+2],u[:,ix[0],ix[1]+2,ix[2]-2],u[:,ix[0],ix[1]-2,ix[2]-2],u[:,ix[0],ix[1]-2,ix[2]+2],
                    u[:,ix[0],ix[1]+1,ix[2]+1],u[:,ix[0],ix[1]+1,ix[2]-1],u[:,ix[0],ix[1]-1,ix[2]-1],u[:,ix[0],ix[1]-1,ix[2]+1]]
    component_yz = make_2d(component_yz)
    
#     uij = np.inner(CenteredFiniteDiffCoeff_offdia,component_xy.T)
    uij = np.dot(CenteredFiniteDiffCoeff_offdia,component_xy)
    
#     print("CenteredFiniteDiffCoeff_offdia.shape: ", CenteredFiniteDiffCoeff_offdia.shape)
#     print("component_xy.T.shape: ", component_xy.T.shape)
#     if CenteredFiniteDiffCoeff_dia.shape[1] != component_xy.shape[0]:
#         component_x = component_x.T

#     if not (np.all(np.dot(CenteredFiniteDiffCoeff_offdia,component_xy) == uij)):
#         print("Dot-inner: ", np.dot(CenteredFiniteDiffCoeff_offdia,component_xy), " --- ", uij)
    
#     uik = np.inner(CenteredFiniteDiffCoeff_offdia,component_xz.T)
    uik = np.dot(CenteredFiniteDiffCoeff_offdia,component_xz)
#     ujk = np.inner(CenteredFiniteDiffCoeff_offdia,component_yz.T)
    ujk = np.dot(CenteredFiniteDiffCoeff_offdia,component_yz)
    return uii,uij,uik,ujj,ujk,ukk


@jit(nopython=True)#, parallel=True)
def forLaplacian(u, ix):
    uii =  np.zeros((u.shape[0],4,4,4))
    ujj =  np.zeros((u.shape[0],4,4,4))
    ukk =  np.zeros((u.shape[0],4,4,4))
    uij =  np.zeros((u.shape[0],4,4,4))
    uik =  np.zeros((u.shape[0],4,4,4))
    ujk =  np.zeros((u.shape[0],4,4,4))
    
    for i in range(4):
        for j in range(4):
            for k in prange(4):
                uii[:,i,j,k], uij[:,i,j,k], uik[:,i,j,k], ujj[:,i,j,k], ujk[:,i,j,k], ukk[:,i,j,k] = HessianNone_Fd4(np.array([ix[0]-1+i,ix[1]-1+j,ix[2]-1+k]),u,dx)

    return uii, ujj, ukk, uij, uik, ujk

# @jit(nopython=True)
def HessianFd4Lag4L(p,u,dx,LW,NB):
    #--------------------------------------------------------
    # p is an np.array(3) containing the three coordinates
    #---------------------------------------------------------
    # get the coefficients
    #----------------------
    ix = p.astype(np.int64)
    fr = p-ix
    gx = LW[int(NB*fr[0])]
    gy = LW[int(NB*fr[1])]
    gz = LW[int(NB*fr[2])]
    #------------------------------------
    # create the 3D kernel from the 
    # outer product of the 1d kernels
    #------------------------------------
    gk = np.einsum('i,j,k',gx,gy,gz)
    #---------------------------------------
    # assemble the 4x4x4 cube and convolve
    #---------------------------------------
#     uii =  np.zeros((u.shape[0],4,4,4))
#     ujj =  np.zeros((u.shape[0],4,4,4))
#     ukk =  np.zeros((u.shape[0],4,4,4))
#     uij =  np.zeros((u.shape[0],4,4,4))
#     uik =  np.zeros((u.shape[0],4,4,4))
#     ujk =  np.zeros((u.shape[0],4,4,4))
    
#     for i in range(4):
#         for j in range(4):
#             for k in range(4):
#                 uii[:,i,j,k], uij[:,i,j,k], uik[:,i,j,k], ujj[:,i,j,k], ujk[:,i,j,k], ukk[:,i,j,k] = HessianNone_Fd4(np.array([ix[0]-1+i,ix[1]-1+j,ix[2]-1+k]),u,dx)
       
    uii,uij,uik,ujj,ujk,ukk = forLaplacian(u, ix)
    
    uii = np.einsum('ijk,lijk->l',gk,uii) #dudxx, dvdxx, dwdxx
    ujj = np.einsum('ijk,lijk->l',gk,ujj) #dudyy, dvdyy, dwdyy
    ukk = np.einsum('ijk,lijk->l',gk,ukk) #dudzz, dvdzz, dwdzz
    
    uij = np.einsum('ijk,lijk->l',gk,uij) #dudxy, dvdxy, dwdxy 
    uik = np.einsum('ijk,lijk->l',gk,uik) #dudxz, dvdxz, dwdxz 
    ujk = np.einsum('ijk,lijk->l',gk,ujk) #dudyz, dvdyz, dwdyz
    
    return uii,uij,uik,ujj,ujk,ukk

In [175]:
# Parameters
dx=2*math.pi/8192
#variable = 'pressure'
variable = 'velocity'
var = get_variable_identifier(variable)
# num_values_per_datapoint = 1 for pressure, 3 for velocity
num_values_per_datapoint = 3
bytes_per_datapoint = 4
timepoint = 0

# line test:
npoints=100
B = np.random.uniform(low=0., high=8192., size=(npoints,3))*dx

# B is the set of array
points = np.floor(B/dx)
points = points.astype(int)

In [176]:
# start interpolation
t1 = time.perf_counter()
NB=100000 # similar with LaginterpLag8C with NB = 1000000
LW_Lag = Lag_looKuptable_4(NB)
###################################################################################
# step one: build unique cornercodes dictionary
###################################################################################

# print(np.array(LW_Lag))
LW_Lag = np.array(LW_Lag) # Ariel added this

# find 8 corner points coordinates for each point
All_eight_corner_points=[]
for point in points:
    x_range = [int(point[0]-3), int(point[0]+4)] ##here is changed 9 points are needed for no interpolation
    y_range = [int(point[1]-3), int(point[1]+4)]
    z_range = [int(point[2]-3), int(point[2]+4)]
    axes_ranges = assemble_axis_data([x_range, y_range, z_range])    
    box = [list(axis_range) for axis_range in axes_ranges]        

    # the order here is how we build bucket later in 'def formbucket'
    full_corner_points=((box[0][0],box[1][0],box[2][0]),(box[0][1],box[1][0],box[2][0]),(box[0][0],box[1][1],box[2][0]),
                   (box[0][1],box[1][1],box[2][0]),(box[0][0],box[1][0],box[2][1]),(box[0][1],box[1][0],box[2][1]),
                   (box[0][0],box[1][1],box[2][1]),(box[0][1],box[1][1],box[2][1]))

    All_eight_corner_points.append(full_corner_points)
    
All_eight_corner_points=tuple(All_eight_corner_points) 

t2 = time.perf_counter()

# find corresponding cornercodes of the 8 corner points for each point
points_cornercodes_pool=[]
points_cornercodes_temp = []
points_cornercodes_all = []

num_voxel = []

for point in All_eight_corner_points:
    points_cornercodes_temp = []
    for pp in point:    
        datapoint = [p % cube.N for p in pp]
        point_cornercode, offset = cube.get_offset(datapoint)
        points_cornercodes_temp.append(point_cornercode)
    # the number of voxel is always 8 for each queried point so that we can build bucket using 'def formBucket' for any case,
    # For point only need one voxel e.g, (3.5,3.5,3.5), points_cornercodes_temp = [0,0,0,0,0,0,0,0], 
    # For point only need two voxel e.g, (7.5,3.5,3.5), points_cornercodes_temp = [0,512,0,512,0,512,0,512]
    # For point only need four voxel e.g, (7.5,7.5,3.5), points_cornercodes_temp = [0,512,1024,1536,0,512,1024,1536]
    # ...for the most of cases, it needs 8 voxels
    # points_cornercodes_temp store the order that build bucket in 'def formBucket' 
    num_voxel.append(len(points_cornercodes_temp))
    points_cornercodes_all.append(points_cornercodes_temp)

# find unique cornercodes for the voxels pool
points_cornercodes_pool = np.unique(list(chain.from_iterable(points_cornercodes_all)))

t3 = time.perf_counter()

# groupe cornercodes in a dictionary: {[path]:[db_minLim]:[cornercodes]}
Voxels_in_pool = {}
for i in range(len(points_cornercodes_pool)):
    cornercode = points_cornercodes_pool[i]
    if i==0:
        db_minLim, db_maxLim, path = findPath(cube, cornercode)
        if path not in Voxels_in_pool:
            Voxels_in_pool[path] = {}
            if db_minLim not in Voxels_in_pool[path]:
                Voxels_in_pool[path][db_minLim] = {}
                Voxels_in_pool[path][db_minLim] = ([cornercode])
    else:
        if cornercode <= db_maxLim:
            Voxels_in_pool[path][db_minLim].append(cornercode)
        else:
            db_minLim, db_maxLim, path = findPath(cube, cornercode)
            if path not in Voxels_in_pool:
                Voxels_in_pool[path] = {}
                if db_minLim not in Voxels_in_pool[path]:
                    Voxels_in_pool[path][db_minLim] = {}
                    Voxels_in_pool[path][db_minLim] = ([cornercode])
                         
t3_1 = time.perf_counter()

# further groupe adjacent cornercodes in the dictionary: {[path]:[db_minLim]:([cornercodes range], number of voxels)}
c = get_constants()
for path in Voxels_in_pool:
    grouped_voxels_pool = []
    for db_minLim in Voxels_in_pool[path]:
        Sorted_voxels = np.sort(Voxels_in_pool[path][db_minLim])
        #glue cornercodes if they are adjacent
        group_voxels = ranges(Sorted_voxels)
        for group_voxel in group_voxels:
            # the number of glued voxels
            number_vox = (group_voxel[1] - group_voxel[0])/512 + 1
            grouped_voxels_pool.append([group_voxel, number_vox])
        
        #update Voxels_in_pool dictionary with glued voxels
        Voxels_in_pool[path][db_minLim]=grouped_voxels_pool    
        
###################################################################################
# step two: reading voxel sequentially and build voxels pool
###################################################################################

t3_2 = time.perf_counter()
# Reading data sequentially
voxel_value_pool = []
for db_file in Voxels_in_pool:
    for db_minLim in Voxels_in_pool[db_file]:
        morton_voxels_to_read = Voxels_in_pool[db_file][db_minLim]
        for morton_data in morton_voxels_to_read:
                morton_index_range = morton_data[0]
                num_voxels = morton_data[1]

                seek_distance = num_values_per_datapoint * bytes_per_datapoint * (morton_index_range[0] - db_minLim)                    

                read_length = num_values_per_datapoint * int(num_voxels) * 512
                
                # read voxels and reshape to (number of voxels, 8, 8, 8) because we read voxels in chunk
                voxel_value = np.reshape(np.fromfile(db_file, dtype = 'f', count = read_length, offset = seek_distance), (int(num_voxels),8,8,8,num_values_per_datapoint))
                
                # build voxel pools
                voxel_value_pool.extend(voxel_value)
    
    

###################################################################################
# step three: Building bucket for each point and calculate interpolation
###################################################################################    
t4 = time.perf_counter() 
# Calculate interpolation for each point
ui_Lag = []
ui_Spline = []
# Loop over all queried points
for j in range(len(points)):
    voxel_point_value = []
    Bucket = []
    center_point = []
    # num_voxel[j] is always 8, because we need 8 voxels
    for i in range(num_voxel[j]):
        # pull out correspoding 8 voxels from the voxels pool
        index = np.where(points_cornercodes_all[j][i]==points_cornercodes_pool)[0][0]
        voxel_point_value.append(voxel_value_pool[index]) 
    # build bucket for each queried point    
    Bucket = formBucket(voxel_point_value,num_values_per_datapoint,Bucket_length = 16)
    # need the center coordinate of the bucket (e.g, (7.5, 7.5, 7.5) for the first bucket) to calculate interpolation.
    center_point = findCenter(B[j], dx)
    ui_Lag.append(HessianFd4Lag4L(center_point,Bucket,dx,LW_Lag,NB))

t5 = time.perf_counter()
# TIC = (t2-t1)
print('Find all 8 corners coordinates',(t2-t1),' sec')
print('Find unique cornercode',(t3-t2),' sec')
print('Load voxel pool',(t4-t3),' sec')
print('Load voxel pool_creat_empty_dictionary',(t3_1-t3),' sec')
print('Load voxel pool_creat_voxel_dictionary',(t3_2-t3_1),' sec')
print('Load voxel pool_read_from dictionary',(t4-t3_2),' sec')
print('Interpolation, Build Bucket + Calc',(t5-t4),' sec')
print('In total',(t5-t1),' sec')

<ipython-input-174-c4ea80cab667>:215: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-174-c4ea80cab667>", line 148:
@jit(nopython=True, parallel=True)
def HessianNone_Fd4(p,u,dx):
^

  uii[:,i,j,k], uij[:,i,j,k], uik[:,i,j,k], ujj[:,i,j,k], ujk[:,i,j,k], ukk[:,i,j,k] = HessianNone_Fd4(np.array([ix[0]-1+i,ix[1]-1+j,ix[2]-1+k]),u,dx)
<ipython-input-174-c4ea80cab667>:215: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  uii[:,i,j,k], uij[:,i,j,k], uik[:,i,j,k], ujj[:,i,j,k], ujk[:,i,j,k], ukk[:,i,j,k] = HessianNone_Fd4(np.array([ix[0]-1+i,ix[1]-1+j,ix[2]-1+k]),u,dx)


TypingError: Failed in nopython mode pipeline (step: nopython mode backend)
[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(float64, 1d, A), UniTuple(int64 x 2), float64)
 
There are 16 candidate implementations:
[1m      - Of which 14 did not match due to:
      Overload of function 'setitem': File: <numerous>: Line N/A.
        With argument(s): '(array(float64, 1d, A), UniTuple(int64 x 2), float64)':[0m
[1m       No match.[0m
[1m      - Of which 2 did not match due to:
      Overload in function 'SetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 171.
        With argument(s): '(array(float64, 1d, A), UniTuple(int64 x 2), float64)':[0m
[1m       Rejected as the implementation raised a specific error:
         TypeError: cannot index array(float64, 1d, A) with 2 indices: UniTuple(int64 x 2)[0m
  raised from /home/idies/miniconda3